# SRnDeblur_joint 1001

## Network

![title](images/SRnDeblurN.png)

In [1]:
import torch
import argparse
import os
import random
import math
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TTF
import torchvision.models as models
from torch.backends import cudnn
from torch import optim
from torch.autograd import Variable
from torch.utils import data
from torchvision import transforms
from torchvision import datasets
from torchvision.utils import make_grid
from PIL import Image
from tensorboardX import SummaryWriter

## Generator

![title](images/generator.PNG)

In [2]:
class Generator(nn.Module):
    def __init__(self, batch_size):
        super(Generator, self).__init__()

        bn = None
        if batch_size == 1:
            bn = False # Instance Normalization
        else:
            bn = True # Batch Normalization

        #============================ upscale ============================#
        self.upscale8 = nn.Sequential(
            # [3x32x32] -> [64x32x32]
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=9, stride=1, padding=4, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # [64x32x32] -> [256x32x32]
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            # [256x32x32] -> [64x64x64]
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, inplace=True),
            # [64x64x64] -> [256x64x64]
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            # [256x64x64] -> [64x128x128]
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, inplace=True),
            # [64x128x128] -> [256x128x128]
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            # [256x128x128] -> [64x256x256]
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, inplace=True),  
            # [64x256x256] -> [3x256x256]
            nn.Conv2d(in_channels=64, out_channels=3, kernel_size=3, stride=1, padding=1, bias=False)
        )
        #============================ upscale ============================#


        # nn.Conv2d(input channel 수, convolution에 의해 생성된 channel 수, kernel size, stride=default 1, padding=default 0)
        # [3x256x256] -> [64x128x128]
        self.conv1 = nn.Conv2d(3, 64, 4, 2, 1)
        # [64x256x256] -> [64x128x128]
#         self.conv1 = nn.Conv2d(64, 64, 3, 1, 1)

        # -> [128x64x64]
        conv2 = [nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1)]
        if bn == True:
            conv2 += [nn.BatchNorm2d(128)]
        else:
            conv2 += [nn.InstanceNorm2d(128)]
        self.conv2 = nn.Sequential(*conv2)

        # -> [256x32x32]
        conv3 = [nn.LeakyReLU(0.2, inplace=True),
                 nn.Conv2d(128, 256, 4, 2, 1)]
        if bn == True:
            conv3 += [nn.BatchNorm2d(256)]
        else:
            conv3 += [nn.InstanceNorm2d(256)]
        self.conv3 = nn.Sequential(*conv3)

        # -> [512x16x16]
        conv4 = [nn.LeakyReLU(0.2, inplace=True),
                 nn.Conv2d(256, 512, 4, 2, 1)]
        if bn == True:
            conv4 += [nn.BatchNorm2d(512)]
        else:
            conv4 += [nn.InstanceNorm2d(512)]
        self.conv4 = nn.Sequential(*conv4)

        # -> [512x8x8]
        conv5 = [nn.LeakyReLU(0.2, inplace=True),
                 nn.Conv2d(512, 512, 4, 2, 1)]
        if bn == True:
            conv5 += [nn.BatchNorm2d(512)]
        else:
            conv5 += [nn.InstanceNorm2d(512)]
        self.conv5 = nn.Sequential(*conv5)

        # -> [512x4x4]
        conv6 = [nn.LeakyReLU(0.2, inplace=True),
                 nn.Conv2d(512, 512, 4, 2, 1)]
        if bn == True:
            conv6 += [nn.BatchNorm2d(512)]
        else:
            conv6 += [nn.InstanceNorm2d(512)]
        self.conv6 = nn.Sequential(*conv6)

        # -> [512x2x2]
        conv7 = [nn.LeakyReLU(0.2, inplace=True),
                 nn.Conv2d(512, 512, 4, 2, 1)]
        if bn == True:
            conv7 += [nn.BatchNorm2d(512)]
        else:
            conv7 += [nn.InstanceNorm2d(512)]
        self.conv7 = nn.Sequential(*conv7)

        # -> [512x1x1]
        conv8 = [nn.LeakyReLU(0.2, inplace=True),
                 nn.Conv2d(512, 512, 4, 2, 1)]
        if bn == True:
            conv8 += [nn.BatchNorm2d(512)]
        else:
            conv8 += [nn.InstanceNorm2d(512)]
        self.conv8 = nn.Sequential(*conv8)

        # -> [512x2x2]
        deconv8 = [nn.ReLU(),
                   nn.ConvTranspose2d(512, 512, 4, 2, 1)]
        if bn == True:
            deconv8 += [nn.BatchNorm2d(512), nn.Dropout(0.5)]
        else:
            deconv8 += [nn.InstanceNorm2d(512), nn.Dropout(0.5)]
        self.deconv8 = nn.Sequential(*deconv8)

        # [(512+512)x2x2] -> [512x4x4]
        deconv7 = [nn.ReLU(),
                   nn.ConvTranspose2d(512 * 2, 512, 4, 2, 1)]
        if bn == True:
            deconv7 += [nn.BatchNorm2d(512), nn.Dropout(0.5)]
        else:
            deconv7 += [nn.InstanceNorm2d(512), nn.Dropout(0.5)]
        self.deconv7 = nn.Sequential(*deconv7)

        # [(512+512)x4x4] -> [512x8x8]
        deconv6 = [nn.ReLU(),
                   nn.ConvTranspose2d(512 * 2, 512, 4, 2, 1)]
        if bn == True:
            deconv6 += [nn.BatchNorm2d(512), nn.Dropout(0.5)]
        else:
            deconv6 += [nn.InstanceNorm2d(512), nn.Dropout(0.5)]
        self.deconv6 = nn.Sequential(*deconv6)

        # [(512+512)x8x8] -> [512x16x16]
        deconv5 = [nn.ReLU(),
                   nn.ConvTranspose2d(512 * 2, 512, 4, 2, 1)]
        if bn == True:
            deconv5 += [nn.BatchNorm2d(512)]
        else:
            deconv5 += [nn.InstanceNorm2d(512)]
        self.deconv5 = nn.Sequential(*deconv5)

        # [(512+512)x16x16] -> [256x32x32]
        deconv4 = [nn.ReLU(),
                   nn.ConvTranspose2d(512 * 2, 256, 4, 2, 1)]
        if bn == True:
            deconv4 += [nn.BatchNorm2d(256)]
        else:
            deconv4 += [nn.InstanceNorm2d(256)]
        self.deconv4 = nn.Sequential(*deconv4)
        
        # [(512+512)x16x16] -> [256x32x32]
        deconv4_0 = [nn.ReLU(),
                   nn.ConvTranspose2d(512 * 1, 256, 4, 2, 1)]
        if bn == True:
            deconv4_0 += [nn.BatchNorm2d(256)]
        else:
            deconv4_0 += [nn.InstanceNorm2d(256)]
        self.deconv4_0 = nn.Sequential(*deconv4_0)        

        # [(256+256)x32x32] -> [128x64x64]
        deconv3 = [nn.ReLU(),
                   nn.ConvTranspose2d(256 * 2, 128, 4, 2, 1)]
        if bn == True:
            deconv3 += [nn.BatchNorm2d(128)]
        else:
            deconv3 += [nn.InstanceNorm2d(128)]
        self.deconv3 = nn.Sequential(*deconv3)

        # [(256+256)x32x32] -> [128x64x64]
        deconv3_0 = [nn.ReLU(),
                   nn.ConvTranspose2d(256 * 2, 128, 4, 2, 1)]
        if bn == True:
            deconv3_0 += [nn.BatchNorm2d(128)]
        else:
            deconv3_0 += [nn.InstanceNorm2d(128)]
        self.deconv3_0 = nn.Sequential(*deconv3_0)
        
        # [(128+128)x64x64] -> [64x128x128]
        deconv2 = [nn.ReLU(),
                   nn.ConvTranspose2d(128 * 2, 64, 4, 2, 1)]
        if bn == True:
            deconv2 += [nn.BatchNorm2d(64)]
        else:
            deconv2 += [nn.InstanceNorm2d(64)]
        self.deconv2 = nn.Sequential(*deconv2)
        
        # [(128+128)x64x64] -> [64x128x128]
        deconv2_0 = [nn.ReLU(),
                   nn.ConvTranspose2d(128 * 2, 64, 4, 2, 1)]
        if bn == True:
            deconv2_0 += [nn.BatchNorm2d(64)]
        else:
            deconv2_0 += [nn.InstanceNorm2d(64)]
        self.deconv2_0 = nn.Sequential(*deconv2_0)

        # [(64+64)x128x128] -> [3x256x256]
        self.deconv1 = nn.Sequential(
            nn.ReLU(),
            nn.ConvTranspose2d(64 * 2, 3, 4, 2, 1),
            nn.Tanh()
        )
        
        # [(64+64)x128x128] -> [3x256x256]
        self.deconv1_0 = nn.Sequential(
            nn.ReLU(),
            nn.ConvTranspose2d(64 * 2, 3, 4, 2, 1),
            nn.Tanh()
        )
        
    def forward(self, x):
        
        upx = self.upscale8(x)
        c1 = self.conv1(upx)
        c2 = self.conv2(c1)
        c3 = self.conv3(c2)
        c4 = self.conv4(c3)
        c5 = self.conv5(c4)
        c6 = self.conv6(c5)
        c7 = self.conv7(c6)
        c8 = self.conv8(c7)
        
        d3_0 = self.deconv4_0(c4)
        d3_0 = torch.cat((c3,d3_0), dim=1)
        d2_0 = self.deconv3_0(d3_0)
        d2_0 = torch.cat((c2,d2_0), dim=1)
        d1_0 = self.deconv2_0(d2_0)
        d1_00 = torch.cat((c1,d1_0), dim=1)
        outLR = self.deconv1_0(d1_00)
        
        d7 = self.deconv8(c8)
        d7 = torch.cat((c7, d7), dim=1)
        d6 = self.deconv7(d7)
        d6 = torch.cat((c6, d6), dim=1)
        d5 = self.deconv6(d6)
        d5 = torch.cat((c5, d5), dim=1)
        d4 = self.deconv5(d5)
        d4 = torch.cat((c4, d4), dim=1)
        d3 = self.deconv4(d4)
        d3 = torch.cat((c3, d3), dim=1)
        d2 = self.deconv3(d3)
        d2 = torch.cat((c2, d2), dim=1)
        d1 = self.deconv2(d2)
        d1 = torch.add(d1,d1_0)
        d1 = torch.cat((c1, d1), dim=1)
        outHR = self.deconv1(d1)
#         output = torch.add(outLR,outHR)
#         d1 = torch.cat((c1, d1), dim=1)
#         outHR = self.deconv1(d1)


#         return outLR, outHR
        return upx,outLR, outHR

## discriminator

![title](images/discriminator.PNG)

In [3]:
class Discriminator(nn.Module):
    def __init__(self, inch, batch_size):
        super(Discriminator, self).__init__()

        bn = None
        if batch_size == 1:
            bn = False  # Instance Normalization
        else:
            bn = True  # Batch Normalization

#         # [(3+3)x256x256] -> [64x128x128] -> [128x64x64]
#         main = [nn.Conv2d(3*2, 64, 4, 2, 1),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Conv2d(64, 128, 4, 2, 1)]
        # [(4+4)x256x256] -> [64x128x128] -> [128x64x64]
        main = [nn.Conv2d(inch, 64, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, 2, 1)]        
        if bn == True:
            main += [nn.BatchNorm2d(128)]
        else:
            main += [nn.InstanceNorm2d(128)]

        # -> [256x32x32]
        main += [nn.LeakyReLU(0.2, inplace=True),
                  nn.Conv2d(128, 256, 4, 2, 1)]
        if bn == True:
            main += [nn.BatchNorm2d(256)]
        else:
            main += [nn.InstanceNorm2d(256)]

        # -> [512x31x31] (Fully Convolutional)
        main += [nn.LeakyReLU(0.2, inplace=True),
                  nn.Conv2d(256, 512, 4, 1, 1)]
        if bn == True:
            main += [nn.BatchNorm2d(512)]
        else:
            main += [nn.InstanceNorm2d(512)]

        # -> [1x30x30] (Fully Convolutional, PatchGAN)
        main += [nn.LeakyReLU(0.2, inplace=True),
                  nn.Conv2d(512, 1, 4, 1, 1)]
                  #nn.Sigmoid()]

        self.main = nn.Sequential(*main)

#     def forward(self, x1, x2): # One for Real, One for Fake
#         out = torch.cat((x1, x2), dim=1)
#         return self.main(out)   
    def forward(self, x):
        return self.main(x)

## VGG for loss

In [4]:
class Vgg19(torch.nn.Module):
    def __init__(self, requires_grad=False):
        super(Vgg19, self).__init__()
        vgg_pretrained_features = models.vgg19(pretrained=True).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        self.slice5 = torch.nn.Sequential()
        for x in range(2):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(2, 7):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(7, 12):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(12, 21):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        for x in range(21, 30):
            self.slice5.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        h_relu1 = self.slice1(X)
        h_relu2 = self.slice2(h_relu1)
        h_relu3 = self.slice3(h_relu2)
        h_relu4 = self.slice4(h_relu3)
        h_relu5 = self.slice5(h_relu4)
        out = [h_relu1, h_relu2, h_relu3, h_relu4, h_relu5]
        return out
    
class VGGLoss(nn.Module):
    def __init__(self):
        super(VGGLoss, self).__init__()
        self.vgg = Vgg19().cuda()
        self.criterion = nn.L1Loss()
        self.weights = [1.0/32, 1.0/16, 1.0/8, 1.0/4, 1.0]

    def forward(self, x, y):
        x_vgg, y_vgg = self.vgg(x), self.vgg(y)
        loss = 0
        for i in range(len(x_vgg)):
            loss += self.weights[i] * self.criterion(x_vgg[i], y_vgg[i].detach())
        return loss

## Helper Function

In [5]:
##### Helper Functions for Data Loading & Pre-processingclass ImageFolder(data.Dataset):
class ImageFolder(data.Dataset):
    def __init__(self, opt):
        # os.listdir function gives all lists of directory
        self.root = opt.dataroot
        self.no_resize_or_crop = opt.no_resize_or_crop
        self.no_flip = opt.no_flip
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
        self.transformM = transforms.Compose([transforms.ToTensor()])
        #=====================================================================================#
        self.dir_A = os.path.join(opt.dataroot,'trainx8')
        self.Aimg_paths = list(map(lambda x:os.path.join(self.dir_A,x),os.listdir(self.dir_A)))
        #=====================================================================================#
#         self.dir_AB = os.path.join(opt.dataroot, 'train')
#         self.image_paths = list(map(lambda x: os.path.join(self.dir_AB, x), os.listdir(self.dir_AB)))
        
    def __getitem__(self, index):
        #=====================================================================================#
        # A : 32x32 (blur + LR)
        # B : 256x256 (LR)
        # C : 256x256 (GT)
        # D : 256x256 (fmask)
        A_path = self.Aimg_paths[index]
        trn = A_path.find('trainx8')
        endn = len(A_path)
        B_path = A_path[:trn]+'wblur'+A_path[trn+7:endn-4]+'.png'
        C_path = A_path[:trn]+'GT'+A_path[trn+7:endn-4]+'.jpg'
        D_path = A_path[:trn]+'fmask'+A_path[trn+7:endn-4]+'.png'
#         B_path = A_path[:trn]+'GT'+A_path[trn+5:endn-4]+'_mask.jpg'
        
        A = Image.open(A_path).convert('RGB')
        B = Image.open(B_path).convert('RGB')
        C = Image.open(C_path).convert('RGB')
        D = Image.open(D_path)
#         A = A.resize((256,256),Image.BICUBIC)
#         B = (C.resize((32,32),Image.BICUBIC)).resize((256,256),Image.BICUBIC)
#         C = C.resize((256,256),Image.BICUBIC)
#         D = D.resize((256,256),Image.BICUBIC)
#         D = D.resize((256,256),Image.BICUBIC)
#         D = torch.zeros(256,256)
#         D = TTF.to_pil_image(D)
                        
        if (not self.no_resize_or_crop):
            A = A.resize((286,286), Image.BICUBIC)
            B = B.resize((286,286), Image.BICUBIC)
            A = self.transform(A)
            B = self.transform(B)
            
            w = int(A.size(2)/2)
            h = A.size(1)
            w_offset = random.randint(0, max(0, w-256-1))
            h_offset = random.randint(0, max(0, h-256-1))
            
            A = A[:, h_offset:h_offset+256, w_offset:w_offset+256]
            B = B[:, h_offset:h_offset+256, w_offset:w_offset+256]
        else:
            A = self.transform(A)
            B = self.transform(B)
            C = self.transform(C)
            D = self.transformM(D)
            A = A[:,:32,:32]
#             A = A[:,:256,:256]
            B = B[:,:256,:256]
            C = C[:,:256,:256]
            D = D[:,:256,:256]
        
#         if (not(self.no_flip)) and random.random() < 0.5:
#             idx = [i for i in range(A.size(2)-1,-1,-1)]
#             idx = torch.LongTensor(idx)
#             A = A.index_select(2,idx)
#             B = B.index_select(2,idx)
#             C = C.index_select(2,idx)
#             D = D.index_select(2,idx)

#         A = (torch.cat((A,D),dim=0))[0:4,:,:]
#         B = (torch.cat((B,C),dim=0))[0:4,:,:]
#         print(A.shape, B.shape, C.shape, D.shape)
#         print('A', A.size())
#         print('B', B.size())
#         print('C', C.size())
#         print('D', D.size())
        return {'A':A, 'B':B, 'C':C, 'D':D}
        
    def __len__(self):
        return len(self.Aimg_paths)

##### Helper Function for GPU Training
def to_variable(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

##### Helper Function for Math
def denorm(x):
    out = (x+1)/2
    return out.clamp(0,1)

##### Helper Functions for GAN Loss (4D Loss Comparison)
def GAN_Loss(input, target, criterion):
    if target == True:
        tmp_tensor = torch.FloatTensor(input.size()).fill_(1.0)
        labels = Variable(tmp_tensor, requires_grad=False)
    else:
        tmp_tensor = torch.FloatTensor(input.size()).fill_(0.0)
        labels = Variable(tmp_tensor, requires_grad=False)
        
    if torch.cuda.is_available():
        labels = labels.cuda()
        
    return criterion(input, labels)


## training argument

In [6]:
# GPU using
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
parser = argparse.ArgumentParser(description='Implementation of SRnDeblur')

# Task
parser.add_argument('--dataroot', required=True, help='path to images (should have subfolders train, val, etc)')
parser.add_argument('--which_direction', type=str, default='AtoB', help='AtoB or BtoA')

# Pre-processing
parser.add_argument('--no_resize_or_crop', action='store_true', help='scaling and cropping of images at load time [resize_and_crop|crop|scale_width|scale_width_and_crop]')
parser.add_argument('--no_flip', action='store_true', help='if specified, do not flip the images for data augmentation')

# Hyper-parameters
parser.add_argument('--num_epochs', type=int, default=100)
parser.add_argument('--batchSize', type=int, default=1, help='input batch size')
parser.add_argument('--lr', type=float, default=0.0002)
parser.add_argument('--beta1', type=float, default=0.5)  # momentum1 in Adam
parser.add_argument('--beta2', type=float, default=0.999)  # momentum2 in Adam
parser.add_argument('--lambda_A', type=float, default=100.0)

# misc
parser.add_argument('--model_path', type=str, default='./models')  # Model Tmp Save
parser.add_argument('--sample_path', type=str, default='./results')  # Results
parser.add_argument('--log_step', type=int, default=10)
parser.add_argument('--sample_step', type=int, default=100)
parser.add_argument('--num_workers', type=int, default=2)

_StoreAction(option_strings=['--num_workers'], dest='num_workers', nargs=None, const=None, default=2, type=<class 'int'>, choices=None, help=None, metavar=None)

## Main

In [7]:
# Pre-settings
cudnn.benchmark = True
global args
args = parser.parse_args(['--dataroot','./datasets/face_SRnDeblur','--which_direction','AtoB',
                          '--num_epochs','1001','--batchSize','16','--no_resize_or_crop',
                          '--log_step','100'])
print(args)

dataset = ImageFolder(args)

data_loader = data.DataLoader(dataset=dataset, batch_size=args.batchSize, shuffle=True, num_workers=args.num_workers)

if not os.path.exists(args.model_path):
    os.makedirs(args.model_path)
if not os.path.exists(args.sample_path):
    os.makedirs(args.sample_path)
    

Namespace(batchSize=16, beta1=0.5, beta2=0.999, dataroot='./datasets/face_SRnDeblur', lambda_A=100.0, log_step=100, lr=0.0002, model_path='./models', no_flip=False, no_resize_or_crop=True, num_epochs=1001, num_workers=2, sample_path='./results', sample_step=100, which_direction='AtoB')


In [9]:
# tensorboardX summary
summary = SummaryWriter('log')
# Networks
# generator = nn.DataParallel(Generator(args.batchSize))
# discriminator = nn.DataParallel(Discriminator(6,args.batchSize))
# discriminatorM = nn.DataParallel(Discriminator(6,args.batchSize))
generator = (Generator(args.batchSize))
discriminatorHR = (Discriminator(6,args.batchSize))
discriminatorHRM = (Discriminator(3,args.batchSize))

dummmy = torch.zeros(1,3,32,32)
summary.add_graph(generator,dummmy)
# summary.add_graph(discriminator,(dummmy,dummmy))

# Losses -> vanilaGAN의 loss, MSE는 LSGAN의 loss
# criterionGAN = nn.BCELoss() 
criterionGAN = nn.MSELoss()
criterionL1 = nn.L1Loss()
vgg_loss = VGGLoss()
# Optimizers
d_parameters = list(discriminatorHR.parameters()) + list(discriminatorHRM.parameters())
g_optimizer = optim.Adam(generator.parameters(), args.lr, [args.beta1, args.beta2])
d_optimizer = optim.Adam(d_parameters, args.lr, [args. beta1, args.beta2])

if torch.cuda.is_available():
    generator = generator.cuda()
    discriminatorHR = discriminatorHR.cuda()
    discriminatorHRM = discriminatorHRM.cuda()
    
### train generator and discriminator
# for printing the log
total_step = len(data_loader)

for epoch  in range(args.num_epochs):
    for i, sample in enumerate(data_loader):
        # A : 32x32 (blur + LR)
        # B : 256x256 (LR)
        # C : 256x256 (GT)
        # D : 256x256 (fmask)
        input_A = sample['A']
        GTLR = sample['B']
        GTHR = sample['C']
        fmaskGT = sample['D'].to("cuda")
        
        #======================== preparing ========================#
        in_blurLR = to_variable(input_A)
        upx, fakeLR, fakeHR = generator(in_blurLR)
        v_GTLR = to_variable(GTLR)
        v_GTHR = to_variable(GTHR)
        fakeHR_md = fakeHR * fmaskGT
        GTHR_md = v_GTHR * fmaskGT
        
        #========================= train D =========================#
        # zero_grad : 역전파 실행 전 변화도 0으로 만듦
        discriminatorHR.zero_grad()
        discriminatorHRM.zero_grad()
        
        pred_fakeHR = discriminatorHR(torch.cat((upx.detach(), fakeHR.detach()),dim=1))
        loss_D_fake = GAN_Loss(pred_fakeHR, False, criterionGAN)
        pred_realHR = discriminatorHR(torch.cat((upx.detach(), v_GTHR),dim=1))
        loss_D_real = GAN_Loss(pred_realHR, True, criterionGAN)
        
        pred_fakeM = discriminatorHRM(fakeHR_md.detach())
        loss_D_fakeM = GAN_Loss(pred_fakeM, False, criterionGAN)
        pred_realM = discriminatorHRM(GTHR_md)
        loss_D_realM = GAN_Loss(pred_realM, True, criterionGAN)
        
        # combined loss
        loss_D = (loss_D_fake + loss_D_real + loss_D_fakeM + loss_D_realM) * 0.25 #0.25 & 0.5
        loss_D.backward()
        d_optimizer.step()
        #========================= train G =========================#
        generator.zero_grad()
        
        pred_fake = discriminatorHR(torch.cat((upx, fakeHR), dim=1))
        pred_fakeM = discriminatorHRM(fakeHR_md)
        loss_G_GAN = GAN_Loss(pred_fake, True, criterionGAN) + GAN_Loss(pred_fakeM, True, criterionGAN)
        
        loss_G_L1_LR = criterionL1(fakeLR, v_GTLR)
        loss_G_L1_HR = criterionL1(fakeHR, v_GTHR)
        loss_G_L1_HRM = criterionL1(fakeHR_md, GTHR_md)
        
        loss_G_vgg_HR = vgg_loss(fakeHR, v_GTHR)
        
        loss_G = loss_G_GAN + (loss_G_L1_LR + loss_G_L1_HR + loss_G_L1_HRM) * args.lambda_A + loss_G_vgg_HR * 10
        
        loss_G.backward()
        g_optimizer.step()
        
        # print the log information
        if (i+1) % args.log_step == 0:
            print('Epoch [%d/%d], BatchStep[%d/%d]' % (epoch + 1, args.num_epochs, i + 1, total_step))
            print('D_loss: %.4f, G_loss: %.4f' % (loss_D.data, loss_G.data))
            print('D_RealHR_loss: %.4f, D_FakeHR_loss: %.4f, D_RealHRM_loss: %.4f, D_FakeHRM_loss: %.4f' 
                  % (loss_D_real.data, loss_D_fake.data, loss_D_realM.data, loss_D_fakeM.data))          
            print('GAN_loss: %.4f, G_L1_LR_loss: %.4f, G_L1_HR_loss: %.4f, G_L1_HRM_loss: %.4f, G_vgg_HR_loss: %.4f '
                  % (loss_G_GAN.data, loss_G_L1_LR.data, loss_G_L1_HR.data,loss_G_L1_HRM.data, loss_G_vgg_HR.data))
            
        # save the sampled images
        if (i+1)%args.sample_step == 0:
            in_Ar = upx[0:4,:,:,:]
            fake_Br = fakeHR[0:4,:,:,:]
            real_Br = v_GTHR[0:4,:,:,:]
            fake_Brr = fakeLR[0:4,:,:,:]
            real_Brr = v_GTLR[0:4,:,:,:]
            
            resHR = torch.cat((torch.cat((in_Ar, fake_Br),dim=2), real_Br), dim=2)
            resLR = torch.cat((torch.cat((in_Ar, fake_Brr),dim=2), real_Brr),dim=2)
            torchvision.utils.save_image(denorm(resHR.data), os.path.join(args.sample_path, 'Generated_HR-%d-%d.png' % (epoch + 1, i + 1)))
            torchvision.utils.save_image(denorm(resLR.data), os.path.join(args.sample_path, 'Generated_LR-%d-%d.png' % (epoch + 1, i + 1)))

#             resMasked_0 = torch.cat((torch.cat((torch.cat((fake_Br,real_Br),dim=2),fmasked_FB[0:4,:,:,:]),dim=2),fmasked_GT[0:4,:,:,:]),dim=2)
#             resMasked = torch.cat((torch.cat((resMasked_0,parmasked_FB[0:4,:,:,:]),dim=2),parmasked_FB[0:4,:,:,:]),dim=2)
#             torchvision.utils.save_image(denorm(resMasked.data), os.path.join(args.sample_path, 'Generated_RM-%d-%d.png' % (epoch + 1, i + 1)))
            
    # save summary
    summary.add_scalar('loss/loss_D_real', loss_D_real.item(), epoch)
    summary.add_scalar('loss/loss_D_fake', loss_D_fake.item(), epoch)
    summary.add_scalar('loss/loss_D_realM', loss_D_realM.item(), epoch)
    summary.add_scalar('loss/loss_D_fakeM', loss_D_fakeM.item(), epoch)
    summary.add_scalar('loss/loss_D', loss_D.item(), epoch)
    
    summary.add_scalar('loss/loss_G_GAN', loss_G_GAN.item(), epoch)
    summary.add_scalar('loss/loss_G_L1_LR', loss_G_L1_LR.item(), epoch)
    summary.add_scalar('loss/loss_G_L1_HR', loss_G_L1_HR.item(), epoch)
    summary.add_scalar('loss/loss_G_L1_HRM', loss_G_L1_HRM.item(), epoch)
    summary.add_scalar('loss/loss_G_vgg_HR', loss_G_vgg_HR.item(), epoch)
    summary.add_scalar('loss/loss_G', loss_G.item(), epoch)
    
    fakeOutHR = make_grid(fake_Br, normalize=True, scale_each=True)
    realGTHR = make_grid(real_Br, normalize=True, scale_each=True)
    fakeOutLR = make_grid(fake_Brr, normalize=True, scale_each=True)
    realGTLR = make_grid(real_Brr, normalize=True, scale_each=True)
    
    summary.add_image('0_GT_HR', realGTHR, epoch)
    summary.add_image('1_generated HR', fakeOutHR, epoch)
    summary.add_image('2_GT_LR', realGTLR, epoch)
    summary.add_image('3_generated LR', fakeOutLR, epoch)
    
    # save the model parameters
    if epoch % 10 == 0:
        g_path = os.path.join(args.model_path, 'generator-%d.pkl' % (epoch + 1))
        torch.save(generator.state_dict(), g_path)

Epoch [1/1001], BatchStep[100/1800]
D_loss: 0.2419, G_loss: 35.7701
D_RealHR_loss: 0.2527, D_FakeHR_loss: 0.2253, D_RealHRM_loss: 0.1698, D_FakeHRM_loss: 0.3200
GAN_loss: 0.7607, G_L1_LR_loss: 0.1040, G_L1_HR_loss: 0.1237, G_L1_HRM_loss: 0.0378, G_vgg_HR_loss: 0.8462 
Epoch [1/1001], BatchStep[200/1800]
D_loss: 0.1739, G_loss: 31.3750
D_RealHR_loss: 0.1660, D_FakeHR_loss: 0.1383, D_RealHRM_loss: 0.1540, D_FakeHRM_loss: 0.2372
GAN_loss: 0.8210, G_L1_LR_loss: 0.0884, G_L1_HR_loss: 0.1075, G_L1_HRM_loss: 0.0326, G_vgg_HR_loss: 0.7703 
Epoch [1/1001], BatchStep[300/1800]
D_loss: 0.1237, G_loss: 32.1130
D_RealHR_loss: 0.0579, D_FakeHR_loss: 0.1409, D_RealHRM_loss: 0.1213, D_FakeHRM_loss: 0.1746
GAN_loss: 1.1973, G_L1_LR_loss: 0.0868, G_L1_HR_loss: 0.1090, G_L1_HRM_loss: 0.0389, G_vgg_HR_loss: 0.7446 
Epoch [1/1001], BatchStep[400/1800]
D_loss: 0.1596, G_loss: 27.8914
D_RealHR_loss: 0.1437, D_FakeHR_loss: 0.0191, D_RealHRM_loss: 0.0968, D_FakeHRM_loss: 0.3789
GAN_loss: 0.8708, G_L1_LR_loss: 

Epoch [2/1001], BatchStep[1400/1800]
D_loss: 0.0648, G_loss: 21.2736
D_RealHR_loss: 0.0202, D_FakeHR_loss: 0.2216, D_RealHRM_loss: 0.0084, D_FakeHRM_loss: 0.0092
GAN_loss: 1.7499, G_L1_LR_loss: 0.0415, G_L1_HR_loss: 0.0718, G_L1_HRM_loss: 0.0226, G_vgg_HR_loss: 0.5933 
Epoch [2/1001], BatchStep[1500/1800]
D_loss: 0.0243, G_loss: 23.6279
D_RealHR_loss: 0.0123, D_FakeHR_loss: 0.0803, D_RealHRM_loss: 0.0031, D_FakeHRM_loss: 0.0015
GAN_loss: 2.0429, G_L1_LR_loss: 0.0462, G_L1_HR_loss: 0.0823, G_L1_HRM_loss: 0.0228, G_vgg_HR_loss: 0.6455 
Epoch [2/1001], BatchStep[1600/1800]
D_loss: 0.0486, G_loss: 22.6562
D_RealHR_loss: 0.0548, D_FakeHR_loss: 0.1274, D_RealHRM_loss: 0.0107, D_FakeHRM_loss: 0.0015
GAN_loss: 1.3993, G_L1_LR_loss: 0.0468, G_L1_HR_loss: 0.0784, G_L1_HRM_loss: 0.0244, G_vgg_HR_loss: 0.6295 
Epoch [2/1001], BatchStep[1700/1800]
D_loss: 0.0711, G_loss: 21.6987
D_RealHR_loss: 0.0781, D_FakeHR_loss: 0.0928, D_RealHRM_loss: 0.0287, D_FakeHRM_loss: 0.0850
GAN_loss: 1.5475, G_L1_LR_lo

Epoch [4/1001], BatchStep[900/1800]
D_loss: 0.0155, G_loss: 21.5168
D_RealHR_loss: 0.0387, D_FakeHR_loss: 0.0194, D_RealHRM_loss: 0.0008, D_FakeHRM_loss: 0.0031
GAN_loss: 1.7533, G_L1_LR_loss: 0.0441, G_L1_HR_loss: 0.0715, G_L1_HRM_loss: 0.0223, G_vgg_HR_loss: 0.5974 
Epoch [4/1001], BatchStep[1000/1800]
D_loss: 0.0465, G_loss: 21.3884
D_RealHR_loss: 0.0347, D_FakeHR_loss: 0.1500, D_RealHRM_loss: 0.0007, D_FakeHRM_loss: 0.0005
GAN_loss: 1.6727, G_L1_LR_loss: 0.0448, G_L1_HR_loss: 0.0736, G_L1_HRM_loss: 0.0210, G_vgg_HR_loss: 0.5779 
Epoch [4/1001], BatchStep[1100/1800]
D_loss: 0.0568, G_loss: 20.8750
D_RealHR_loss: 0.1136, D_FakeHR_loss: 0.1126, D_RealHRM_loss: 0.0007, D_FakeHRM_loss: 0.0003
GAN_loss: 1.6258, G_L1_LR_loss: 0.0400, G_L1_HR_loss: 0.0723, G_L1_HRM_loss: 0.0214, G_vgg_HR_loss: 0.5881 
Epoch [4/1001], BatchStep[1200/1800]
D_loss: 0.0767, G_loss: 19.5971
D_RealHR_loss: 0.2657, D_FakeHR_loss: 0.0402, D_RealHRM_loss: 0.0005, D_FakeHRM_loss: 0.0004
GAN_loss: 1.3539, G_L1_LR_los

Epoch [6/1001], BatchStep[400/1800]
D_loss: 0.0315, G_loss: 20.8816
D_RealHR_loss: 0.0210, D_FakeHR_loss: 0.0845, D_RealHRM_loss: 0.0167, D_FakeHRM_loss: 0.0037
GAN_loss: 1.7698, G_L1_LR_loss: 0.0411, G_L1_HR_loss: 0.0712, G_L1_HRM_loss: 0.0200, G_vgg_HR_loss: 0.5883 
Epoch [6/1001], BatchStep[500/1800]
D_loss: 0.1949, G_loss: 20.4360
D_RealHR_loss: 0.0366, D_FakeHR_loss: 0.1880, D_RealHRM_loss: 0.2758, D_FakeHRM_loss: 0.2794
GAN_loss: 1.6445, G_L1_LR_loss: 0.0411, G_L1_HR_loss: 0.0702, G_L1_HRM_loss: 0.0206, G_vgg_HR_loss: 0.5607 
Epoch [6/1001], BatchStep[600/1800]
D_loss: 0.2594, G_loss: 20.6977
D_RealHR_loss: 0.0237, D_FakeHR_loss: 0.2463, D_RealHRM_loss: 0.7466, D_FakeHRM_loss: 0.0211
GAN_loss: 1.4315, G_L1_LR_loss: 0.0417, G_L1_HR_loss: 0.0707, G_L1_HRM_loss: 0.0198, G_vgg_HR_loss: 0.6044 
Epoch [6/1001], BatchStep[700/1800]
D_loss: 0.0662, G_loss: 21.2137
D_RealHR_loss: 0.1924, D_FakeHR_loss: 0.0124, D_RealHRM_loss: 0.0453, D_FakeHRM_loss: 0.0145
GAN_loss: 1.6361, G_L1_LR_loss: 

GAN_loss: 1.7418, G_L1_LR_loss: 0.0387, G_L1_HR_loss: 0.0667, G_L1_HRM_loss: 0.0214, G_vgg_HR_loss: 0.5581 
Epoch [7/1001], BatchStep[1700/1800]
D_loss: 0.1100, G_loss: 20.6012
D_RealHR_loss: 0.2418, D_FakeHR_loss: 0.0323, D_RealHRM_loss: 0.1553, D_FakeHRM_loss: 0.0105
GAN_loss: 1.5324, G_L1_LR_loss: 0.0413, G_L1_HR_loss: 0.0725, G_L1_HRM_loss: 0.0182, G_vgg_HR_loss: 0.5868 
Epoch [7/1001], BatchStep[1800/1800]
D_loss: 0.0337, G_loss: 21.6296
D_RealHR_loss: 0.0529, D_FakeHR_loss: 0.0591, D_RealHRM_loss: 0.0121, D_FakeHRM_loss: 0.0109
GAN_loss: 1.7567, G_L1_LR_loss: 0.0422, G_L1_HR_loss: 0.0752, G_L1_HRM_loss: 0.0225, G_vgg_HR_loss: 0.5890 
Epoch [8/1001], BatchStep[100/1800]
D_loss: 0.0514, G_loss: 20.7185
D_RealHR_loss: 0.1583, D_FakeHR_loss: 0.0262, D_RealHRM_loss: 0.0157, D_FakeHRM_loss: 0.0054
GAN_loss: 1.3404, G_L1_LR_loss: 0.0408, G_L1_HR_loss: 0.0745, G_L1_HRM_loss: 0.0202, G_vgg_HR_loss: 0.5823 
Epoch [8/1001], BatchStep[200/1800]
D_loss: 0.1155, G_loss: 19.4941
D_RealHR_loss: 

...

Epoch [258/1001], BatchStep[1000/1800]
D_loss: 0.0191, G_loss: 17.0975
D_RealHR_loss: 0.0497, D_FakeHR_loss: 0.0266, D_RealHRM_loss: 0.0001, D_FakeHRM_loss: 0.0000
GAN_loss: 1.9737, G_L1_LR_loss: 0.0322, G_L1_HR_loss: 0.0586, G_L1_HRM_loss: 0.0149, G_vgg_HR_loss: 0.4557 
Epoch [258/1001], BatchStep[1100/1800]
D_loss: 0.0081, G_loss: 15.5940
D_RealHR_loss: 0.0233, D_FakeHR_loss: 0.0088, D_RealHRM_loss: 0.0001, D_FakeHRM_loss: 0.0000
GAN_loss: 1.8178, G_L1_LR_loss: 0.0277, G_L1_HR_loss: 0.0532, G_L1_HRM_loss: 0.0147, G_vgg_HR_loss: 0.4224 
Epoch [258/1001], BatchStep[1200/1800]
D_loss: 0.0063, G_loss: 17.0828
D_RealHR_loss: 0.0132, D_FakeHR_loss: 0.0119, D_RealHRM_loss: 0.0000, D_FakeHRM_loss: 0.0000
GAN_loss: 2.0279, G_L1_LR_loss: 0.0331, G_L1_HR_loss: 0.0572, G_L1_HRM_loss: 0.0161, G_vgg_HR_loss: 0.4418 
Epoch [258/1001], BatchStep[1300/1800]
D_loss: 0.0323, G_loss: 16.2570
D_RealHR_loss: 0.0686, D_FakeHR_loss: 0.0607, D_RealHRM_loss: 0.0000, D_FakeHRM_loss: 0.0000
GAN_loss: 1.6159, G_

Epoch [260/1001], BatchStep[500/1800]
D_loss: 0.0143, G_loss: 16.0118
D_RealHR_loss: 0.0332, D_FakeHR_loss: 0.0240, D_RealHRM_loss: 0.0000, D_FakeHRM_loss: 0.0000
GAN_loss: 1.8492, G_L1_LR_loss: 0.0309, G_L1_HR_loss: 0.0545, G_L1_HRM_loss: 0.0145, G_vgg_HR_loss: 0.4171 
Epoch [260/1001], BatchStep[600/1800]
D_loss: 0.0129, G_loss: 15.7932
D_RealHR_loss: 0.0135, D_FakeHR_loss: 0.0378, D_RealHRM_loss: 0.0001, D_FakeHRM_loss: 0.0001
GAN_loss: 1.8945, G_L1_LR_loss: 0.0300, G_L1_HR_loss: 0.0531, G_L1_HRM_loss: 0.0149, G_vgg_HR_loss: 0.4103 
Epoch [260/1001], BatchStep[700/1800]
D_loss: 0.0123, G_loss: 17.3901
D_RealHR_loss: 0.0212, D_FakeHR_loss: 0.0094, D_RealHRM_loss: 0.0167, D_FakeHRM_loss: 0.0020
GAN_loss: 1.7320, G_L1_LR_loss: 0.0339, G_L1_HR_loss: 0.0611, G_L1_HRM_loss: 0.0152, G_vgg_HR_loss: 0.4631 
Epoch [260/1001], BatchStep[800/1800]
D_loss: 0.0150, G_loss: 17.1108
D_RealHR_loss: 0.0189, D_FakeHR_loss: 0.0409, D_RealHRM_loss: 0.0001, D_FakeHRM_loss: 0.0000
GAN_loss: 1.9984, G_L1_L

Epoch [261/1001], BatchStep[1800/1800]
D_loss: 0.0069, G_loss: 16.0280
D_RealHR_loss: 0.0147, D_FakeHR_loss: 0.0128, D_RealHRM_loss: 0.0000, D_FakeHRM_loss: 0.0000
GAN_loss: 2.0935, G_L1_LR_loss: 0.0300, G_L1_HR_loss: 0.0527, G_L1_HRM_loss: 0.0149, G_vgg_HR_loss: 0.4177 
Epoch [262/1001], BatchStep[100/1800]
D_loss: 0.0089, G_loss: 16.4816
D_RealHR_loss: 0.0257, D_FakeHR_loss: 0.0100, D_RealHRM_loss: 0.0000, D_FakeHRM_loss: 0.0000
GAN_loss: 1.8474, G_L1_LR_loss: 0.0313, G_L1_HR_loss: 0.0553, G_L1_HRM_loss: 0.0173, G_vgg_HR_loss: 0.4246 
Epoch [262/1001], BatchStep[200/1800]
D_loss: 0.0237, G_loss: 16.0648
D_RealHR_loss: 0.0137, D_FakeHR_loss: 0.0810, D_RealHRM_loss: 0.0000, D_FakeHRM_loss: 0.0000
GAN_loss: 2.0109, G_L1_LR_loss: 0.0295, G_L1_HR_loss: 0.0542, G_L1_HRM_loss: 0.0148, G_vgg_HR_loss: 0.4206 
Epoch [262/1001], BatchStep[300/1800]
D_loss: 0.0433, G_loss: 17.2210
D_RealHR_loss: 0.0406, D_FakeHR_loss: 0.1324, D_RealHRM_loss: 0.0000, D_FakeHRM_loss: 0.0001
GAN_loss: 1.9828, G_L1_

Traceback (most recent call last):
  File "/home/juyun/anaconda3/envs/DL/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/juyun/anaconda3/envs/DL/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/juyun/anaconda3/envs/DL/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/juyun/anaconda3/envs/DL/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 